# 00 Deep Q agent with full experimentation tracking

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Paulescu/hands-on-rl/blob/main/04_cart_pole_tune_hparams_like_a_pro/notebooks/00_deep_q_agent_experimentation_with_weights_and_biases.ipynb)

#### 👉 Deep RL is complex, because it's extreme sensitivity to the hyper-parameters.

#### 👉 To develop, debug and optimize RL agents you need to experiment a lot. And to experiment well, you need to use the right tool.

#### 👉My favourite tool is [Weights&Biases]().

<!-- <img src="https://lever-client-logos.s3.amazonaws.com/bb006941-a5fe-4d4c-b13d-931f9b9c303f-1569362661885.png" width="500" height="400" /> -->

#### 👉 Let's train a Deep Q agent to solve the `Cart Pole` environment, with FULL tracking.

![nn](https://raw.githubusercontent.com/Paulescu/hands-on-rl/4c8e064d56c22828c97aa2e3f773050fa36d7d6f/04_cart_pole_tune_hparams_like_a_pro/images/deep_q_net.svg)

## Setup when running in Colab

In [ ]:
!git clone https://github.com/Paulescu/hands-on-rl.git
    
# navigate to atalaia directory
%cd /content/hands-on-rl/04_cart_pole_tune_hparams_like_a_pro

# get modifications made on the repo
!git pull origin master

# install packages requirements
!pip install -r requirements.txt

# install package
!python setup.py install

In [ ]:
import sys
sys.path.append('/content/hands-on-rl/04_cart_pole_tune_hparams_like_a_pro')

In [10]:
%load_ext autoreload
%autoreload 2
%pylab inline
%config InlineBackend.figure_format = 'svg'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


## Environment 🌎

In [2]:
import gym
env = gym.make('CartPole-v1')

## Log in to your W&B account

In [3]:
import wandb

wandb.login()

wandb: Currently logged in as: paulescu (use `wandb login --relogin` to force relogin)


True

## Start a W&B run

In [ ]:
run = wandb.init(
    project="deep-q-learning-hyperparameters",
    entity="paulescu"
)

## Hyperparameters

In [5]:
# Good hyper-parameters
# make you feel great!
hparams = {
    'learning_rate': 0.00016151809562265122,
    'discount_factor': 0.99,
    'batch_size': 32,
    'memory_size': 10000,
    'freq_steps_train': 8,
    'freq_steps_update_target': 10,
    'n_steps_warm_up_memory': 1000,
    'n_gradient_steps': 16,
    'nn_hidden_layers': [256, 256],
    'max_grad_norm': 10,
    'normalize_state': False,
    'epsilon_start': 0.9,
    'epsilon_end': 0.14856584122699473,
    'steps_epsilon_decay': 10000,
}

# SEED = 2386916045
SEED = 0

### Log hyperparameters

In [6]:
wandb.config = hparams.copy()
wandb.config.update({"seed": SEED})

## ⚠️ Fix random seeds to ensure reproducible runs

In [7]:
from src.utils import set_seed
set_seed(env, SEED)

## Deep Q-Agent

In [8]:
from src.q_agent import QAgent
agent = QAgent(env, **hparams, run=wandb) #run=run)

67,586 parameters


## Train the agent 🏋️

In [9]:
from src.loops import train
train(agent, env, n_episodes=500, run=run)

100%|██████████████████████████████████████████████████████████████████████████| 500/500 [24:15<00:00,  2.91s/it]


### Log PyTorch model

In [101]:
# log PyTorch model
# agent.save_checkpoint()

## Evaluate the agent ⏱️

In [11]:
from src.loops import evaluate
rewards, steps = evaluate(
    agent, env,
    n_episodes=1000,
    epsilon=0.00
)

import numpy as np
reward_avg = np.array(rewards).mean()
reward_std = np.array(rewards).std()
print(f'Reward average {reward_avg:.2f}, std {reward_std:.2f}')

100%|████████████████████████████████████████████████████████████████████████| 1000/1000 [01:58<00:00,  8.43it/s]

Reward average 306.13, std 218.31


### Log evaluation metrics

In [16]:
# log eval metrics
# run['eval/reward_avg'] = reward_avg
# run['eval/reward_std'] = reward_std

wandb.log({'eval/reward_avg': reward_avg})
wandb.log({'eval/reward_std': reward_std})

## Plot evaluation rewards

In [15]:
import matplotlib.pyplot as plt
import pandas as pd

fig, ax = plt.subplots(figsize = (10, 4))
ax.set_title("Rewards")    
pd.Series(rewards).plot(kind='hist', bins=100)

wandb.log({"chart": plt})

plt.show()

### Log plot

In [14]:

# run['eval_rewards'].upload(fig)

ValueError: min() arg is an empty sequence

## End of the experiment

In [17]:
run.finish()

eval/reward_avg,▁▁
eval/reward_std,▁▁
train/epsilon,███▇▇▇▇▆▆▅▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▁▁▂▃▁▄▃█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
train/replay_memory_size,▁▁▁▂▂▂▂▃▃▄▅▅████████████████████████████
train/reward,▂▂▁▁▁▁▁▃▂▃▃▃▄▅▂▂▂▂▁▂▁▃▃▄▃███▂██▅██▂▁███▂
train/steps,▂▂▁▁▁▁▁▃▂▃▃▃▄▅▂▂▂▂▁▂▁▃▃▄▃███▂██▅██▂▁███▂
eval/reward_avg,306.135
eval/reward_std,218.30887
train/epsilon,0.14857
train/loss,0.02736
